## 这是一个练习模板，其中包含了 `example` 的大部分代码，但需要您动手修改和补充，以完成对 `v_bar` 数据的超分辨率任务。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import time

## 超参数与配置

In [ ]:
class Config:
    # --- 数据与保存路径 ---
    # 数据集所在的根目录
    DATA_DIR = "data/v_bar"  
    # 模型和可视化结果的保存目录
    SAVE_DIR = "./sr_cnn_results_vbar_demo"

    # --- 文件名模板 ---
    LQ_TEMPLATE = "ubar_sr_input_{split}.npy"
    GT_TEMPLATE = "ubar_hr_{split}.npy"
    META_FILES = {
        "mask": "mask.npy",
        "lat": "lat.npy",
        "lon": "lon.npy"
    }
    
    # --- 训练参数 ---
    BATCH_SIZE = 16
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 1  #自行调整
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # --- 模型参数 ---
    IN_CHANNELS = 1   
    OUT_CHANNELS = 1
    HIDDEN_DIMS = 64

# 创建保存目录，如果不存在
os.makedirs(Config.SAVE_DIR, exist_ok=True)

## 数据准备

In [ ]:
class Dataset(Dataset):
    """
    为海洋超分任务定制的PyTorch Dataset。
    直接加载预先分割好的 low-resolution (lq) 和 high-resolution (gt) 数据。
    """
    def __init__(self, data_dir, lq_filename, gt_filename):
        """
        Args:
            data_dir (str): 数据文件所在的目录。
            lq_filename (str): 低分辨率数据的文件名。
            gt_filename (str): 高分辨率数据的文件名。
        """
        try:
            lq_path = os.path.join(data_dir, lq_filename)
            gt_path = os.path.join(data_dir, gt_filename)
            
            # 加载数据并转换为 float32
            self.lq_data = np.load(lq_path).astype(np.float32)
            self.gt_data = np.load(gt_path).astype(np.float32)

            print(f"成功加载数据: {lq_filename} (shape: {self.lq_data.shape}) 和 {gt_filename} (shape: {self.gt_data.shape})")

        except FileNotFoundError as e:
            raise FileNotFoundError(f"错误: 数据文件未找到。请确保路径 '{data_dir}' 正确且包含所需文件。\n{e}")

    def __len__(self):
        return self.lq_data.shape[0]
    
    def __getitem__(self, idx):
        # 从 (H, W) 扩展为 (C, H, W) 以符合CNN输入要求
        lq_sample = torch.from_numpy(self.lq_data[idx]).unsqueeze(0)
        gt_sample = torch.from_numpy(self.gt_data[idx]).unsqueeze(0)
        
        return {'lq': lq_sample, 'gt': gt_sample}

## CNN 模型定义

In [ ]:
class SimpleSRCNN(nn.Module):
    """一个用于超分辨率的基础CNN模型"""
    def __init__(self, in_channels, out_channels, hidden_dim):
        super(SimpleSRCNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim, out_channels, kernel_size=3, padding=1)
        )
        
    def forward(self, x):
        """
        前向传播
        Args:
            x (torch.Tensor): 输入的低分辨率图像, shape (B, C_in, H, W)
        Returns:
            torch.Tensor: 输出的超分辨率图像, shape (B, C_out, H, W)
        """
        return self.network(x)


## 工具函数

In [ ]:
def calculate_relative_l2_error(pred, gt, mask):
    """
    计算在有效区域（mask内）的相对L2误差。
    公式: ||pred - gt||_2 / ||gt||_2
    
    Args:
        pred (torch.Tensor): 模型的预测结果。
        gt (torch.Tensor): 真实的高分辨率数据。
        mask (torch.Tensor): 布尔掩码，True表示有效计算区域。
        
    Returns:
        float: 计算出的相对L2误差。
    """
    # 确保只在有效区域计算
    pred_masked = pred[mask]
    gt_masked = gt[mask]
    
    error_norm = torch.linalg.norm(pred_masked - gt_masked)
    gt_norm = torch.linalg.norm(gt_masked)
    
    # 防止除以零
    if gt_norm == 0:
        return float('inf') if error_norm > 0 else 0.0
        
    return (error_norm / gt_norm).item()

def plot_results(lq, gt, sr, mask, lon, lat, save_path):
    """
    严谨地可视化lq, gt, sr以及它们与gt之间的误差。
    
    -  颜色方案与示例对齐 (RdBu_r)，前三张图共享色标。
    -  图像设置为正方形长宽比。
    -  除了保存文件，也会显示图像。
    - 五张子图，每张图都有独立的colorbar。
    - 使用经纬度作为坐标轴。
    - 应用mask，将无效区域（陆地）显示为灰色。
    """
    lq = np.squeeze(lq) # 形状变为 (H, W)
    gt = np.squeeze(gt)
    sr = np.squeeze(sr)

    # 将mask外的区域设置为NaN以便于绘图忽略
    lq[~mask] = np.nan
    gt[~mask] = np.nan
    sr[~mask] = np.nan

    error_sr = sr - gt
    error_lq = lq - gt
    error_sr[~mask] = np.nan
    error_lq[~mask] = np.nan

    fig, axes = plt.subplots(1, 5, figsize=(35, 8), dpi=150)
    
    titles = [
        'Low-Resolution Input (LQ)', 'High-Resolution Ground Truth (GT)', 'Super-Resolution Output (SR)',
        'Error (SR - GT)', 'Error (LQ - GT)'
    ]
    data_maps = [lq, gt, sr, error_sr, error_lq]
    
    data_vmin = np.nanmin((lq, gt, sr))
    data_vmax = np.nanmax((lq, gt, sr))
    
    for i, ax in enumerate(axes):
        data = data_maps[i]
        cmap = 'RdBu_r'
        
        # 为数据图和误差图设定不同的颜色范围逻辑
        if 'Error' in titles[i]:
            # 误差图：色条应以0为中心对称
            vmax_abs = np.nanmax(np.abs(data))
            vmin, vmax = -vmax_abs, vmax_abs
        else:
            # 数据图：使用统一的颜色范围
            vmin, vmax = data_vmin, data_vmax
            
        # 使用 pcolormesh 以正确处理经纬度坐标
        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax, shading='auto')
        
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.ax.tick_params(labelsize=12)
        
        ax.set_title(titles[i], fontsize=16, pad=12)
        ax.set_xlabel('Longitude', fontsize=14)
        ax.set_ylabel('Latitude', fontsize=14)
        ax.tick_params(axis='both', which='major', labelsize=12)
        
        # 设置背景色为灰色，代表陆地
        ax.set_facecolor('0.75')
        
        ax.set_aspect('equal', adjustable='box')
    
    plt.tight_layout(pad=2.5)
    plt.savefig(save_path)
    print(f"可视化结果已保存至: {save_path}")
    plt.show()
    
    plt.close(fig)
    
def save_predictions_as_npy(model, loader, mask, save_dir, num_samples=3):
    """
    使用模型对验证集中的样本进行预测，并将指定数量的结果保存为.npy文件。同时保存对应的lq和gt文件
    陆地区域的值将被设置为 NaN。
    """
    print(f"\n--- 正在保存 {num_samples} 个验证集预测结果为 .npy 文件 ---")
    model.eval()
    
    # 创建一个子目录来存放 .npy 文件
    output_dir = os.path.join(save_dir, "validation_npy_predictions")
    os.makedirs(output_dir, exist_ok=True)
    
    saved_count = 0
    with torch.no_grad():
        for batch in loader:
            if saved_count >= num_samples:
                break
                
            lq_tensor = batch['lq'].to(Config.DEVICE)
            gt_tensor = batch['gt'].to(Config.DEVICE) # gt也需要用来保存
            sr_tensor = model(lq_tensor)
            
            # 在一个批次中处理每个样本，直到保存够数量
            for i in range(sr_tensor.shape[0]):
                if saved_count >= num_samples:
                    break
                
                # 获取单个样本，移至CPU并转换为numpy
                lq_np = lq_tensor[i].squeeze().cpu().numpy()
                gt_np = gt_tensor[i].squeeze().cpu().numpy()
                sr_np = sr_tensor[i].squeeze().cpu().numpy()
                
                # 应用mask，将陆地区域设置为 NaN
                lq_np[~mask] = np.nan
                gt_np[~mask] = np.nan
                sr_np[~mask] = np.nan
                
                # 创建文件名（使用虚拟日期）
                date_str = f"2018-01-{saved_count + 1:02d}"
                
                save_path_lq = os.path.join(output_dir, f"{date_str}_lq.npy")
                save_path_gt = os.path.join(output_dir, f"{date_str}_gt.npy")
                save_path_sr = os.path.join(output_dir, f"{date_str}_sr.npy")
                
                np.save(save_path_lq, lq_np)
                np.save(save_path_gt, gt_np)
                np.save(save_path_sr, sr_np)
                # 保存文件
                print(f"  已保存 {date_str} 对应的 lq, gt, sr 文件。")
                
                saved_count += 1
    print("--------------------------------------------------")

## 训练验证

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, mask_tensor, device):
    model.train()
    pass

def validate(model, loader, criterion, mask_tensor, device):
    model.eval()
    pass

## 执行流程

In [ ]:
def main():
    """主函数，执行完整的训练、评估和可视化流程。"""
    
    pass

if __name__ == '__main__':
    main()